In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#www.kaggle.com/datasets/ashkanzarkhah/documents

PATH = "/kaggle/input/document/"

/kaggle/input/document/final.csv


In [3]:
#Importing Libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 

In [4]:
#Reading gathered data
data = pd.read_csv(PATH + 'final.csv')

print(data)

     Unnamed: 0                                               Poem        Poet
0             0  اَلا یا اَیُّهَا السّاقی اَدِرْ کَأسَاً و ناوِ...        حافظ
1             1  دل می‌رود ز دستم صاحب‌دلان خدا را\n\n\nدردا که...        حافظ
2             2  رونق عهد شباب است دگر بُستان را\n\n\nمی‌رسد مژ...        حافظ
3             3  ای فروغِ ماهِ حُسن، از روی رخشان شما\n\n\nآبرو...        حافظ
4             4  خَمی که ابروی شوخِ تو در کمان انداخت\n\n\nبه ق...        حافظ
..          ...                                                ...         ...
295         145  پیرم و آرزویِ وصلِ جوانان دارم\n\nخانه ویران ب...  ایرج میرزا
296         146  آزرده ام از آن بُتِ بسیار ناز کن\n\nپا از گلیم...  ایرج میرزا
297         147  یکی خِرس بودست در جنگلی\n\nدَرَنده هَیُونی قوی...  ایرج میرزا
298         148  بود شیری به بیشه‌ای خفته\n\nموشکی کرد خوابش آش...  ایرج میرزا
299         149  برزگری کِشته خود را دِرود\n\nتا چه خود از بَدو...  ایرج میرزا

[300 rows x 3 columns]


In [5]:
#Checking for missing values
print('missing values stats')
print(data.isnull().sum(), '\n')

missing values stats
Unnamed: 0    0
Poem          0
Poet          0
dtype: int64 



In [6]:
#Removing id column
data = data[['Poem','Poet']]

In [7]:
#Tokenizing documents for counting each word's tokens
data['poem_len_by_words'] = data['Poem'].apply(lambda t: len(hazm.word_tokenize(t)))

In [8]:
#Comparing document\s lengths
min_max_len = data["poem_len_by_words"].min(), data["poem_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 229 	Max: 802


In [9]:
#Defining a function to clean and normalizie our data
#From unnecessary word and characters
def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=True,
        no_digits=True,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)

    # removing extra spaces, hashtags
    text = re.sub("\s+", " ", text)

    return text

In [10]:
#Cleaning our documents
data['cleaned_poem'] = data['Poem'].apply(cleaning)

In [11]:
#Document sample after cleaning
print(data.iloc[2][3])

رونق عهد شباب است دگر بستان را می‌رسد مژده گل بلبل خوش‌الحان را‌ای صبا‌گر به جوانان چمن باز رسی خدمت ما برسان سرو و گل و ریحان را‌گر چنین جلوه کند مغبچه باده‌فروش خاک‌روب در میخانه کنم مژگان را‌ای که بر مه کشی از عنبر سارا چوگان مضطرب‌حال مگردان، من سرگردان را ترسم این قوم که بر دردکشان می‌خندند در سر کار خرابات کنند ایمان را یار مردان خدا باش که در کشتی نوح هست خاکی که به آبی نخرد طوفان را برو از خانه گردون به در و نان مطلب کآن سیه‌کاسه در آخر بکشد مهمان را هر که را خوابگه آخر مشتی خاک است گو چه حاجت که به افلاک کشی ایوان را ماه کنعانی من! مسند مصر آن تو شد وقت آن است که بدرود کنی زندان را حافظا می‌خور و رندی کن و خوش باش ولی دام تزویر مکن چون دگران قرآن را دوش از مسجد سوی میخانه آمد پیر ما چیست یاران طریقت بعد از این تدبیر ما ما مریدان روی سوی قبله چون آریم چون روی سوی خانه خمار دارد پیر ما در خرابات طریقت ما به هم منزل شویم کاین‌چنین رفته است در عهد ازل تقدیر ما عقل اگر داند که دل در بند زلفش چون خوش است عاقلان دیوانه گردند از پی زنجیر ما روی خوبت آیتی از لطف بر ما کشف کرد زان زمان 

/tmp/ipykernel_34/3705850641.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(data.iloc[2][3])


In [12]:
#Document sample before cleaning
print(data.iloc[2][0])

رونق عهد شباب است دگر بُستان را


می‌رسد مژدهٔ گل بلبل خوش‌الحان را


ای صبا گر به جوانان چمن باز رَسی


خدمت ما برسان سرو و گل و ریحان را


گر چنین جلوه کند مغبچهٔ باده‌فروش


خاک‌روبِ درِ میخانه کنم مژگان را


ای که بر مه کشی از عَنبرِ سارا چوگان


مضطرب‌حال مگردان، من سرگردان را


ترسم این قوم که بر دُردکشان می‌خندند


در سر کار خرابات کنند ایمان را


یار مردان خدا باش که در کشتی نوح


هست خاکی که به آبی نخرد طوفان را


برو از خانهٔ گردون به در و نان مطلب


کآن سیه‌کاسه در آخر بِکُشد مهمان را


هر که را خوابگه آخر مشتی خاک است


گو چه حاجت که به افلاک کشی ایوان را


ماه کنعانی من! مسند مصر آنِ تو شد


وقت آن است که بدرود کنی زندان را


حافظا می خور و رندی کن و خوش باش ولی


دام تزویر مکن چون دگران قرآن را


دوش از مسجد سوی میخانه آمد پیر ما


چیست یاران طریقت بعد از این تدبیر ما


ما مریدان روی سوی قبله چون آریم چون


روی سوی خانهٔ خَمّار دارد پیر ما


در خرابات طریقت ما به هم منزل شویم


کاین‌چنین رفته‌ است در عهد ازل تقدیر ما


عقل اگر داند که دل در بند زلفش چون خوش است


عاقلان د

/tmp/ipykernel_34/2168550124.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(data.iloc[2][0])


In [13]:
#Saving our data to backup our processess till here
#In case our kernel dies
old_data = data.copy()
data = data[['cleaned_poem', 'Poet']]
data.to_csv('/kaggle/working/cleaned.csv')

In [16]:
#Defining general configurations for fine tuning our first model
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EEVERY_EPOCH = 1000
CLIP = 0.0
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = 'pytorch_model.bin'

In [17]:
#Mapping lables (authors) and ids together
label2id = {label: i for i, label in enumerate(data.iloc[30*j][1] for j in range(10))}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'حافظ': 0, 'سعدی': 1, 'مولوی': 2, 'ناصرخسرو': 3, 'عطار': 4, 'فردوسی': 5, 'پروین اعتصامی': 6, 'خاقانی': 7, 'شهریار': 8, 'ایرج میرزا': 9}
id2label: {0: 'حافظ', 1: 'سعدی', 2: 'مولوی', 3: 'ناصرخسرو', 4: 'عطار', 5: 'فردوسی', 6: 'پروین اعتصامی', 7: 'خاقانی', 8: 'شهریار', 9: 'ایرج میرزا'}


/tmp/ipykernel_34/2002318297.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label2id = {label: i for i, label in enumerate(data.iloc[30*j][1] for j in range(10))}


In [18]:
#Loading our model's tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u062d\u0627\u0641\u0638",
    "1": "\u0633\u0639\u062f\u06cc",
    "2": "\u0645\u0648\u0644\u0648\u06cc",
    "3": "\u0646\u0627\u0635\u0631\u062e\u0633\u0631\u0648",
    "4": "\u0639\u0637\u0627\u0631",
    "5": "\u0641\u0631\u062f\u0648\u0633\u06cc",
    "6": "\u067e\u0631\u0648\u06cc\u0646 \u0627\u0639\u062a\u0635\u0627\u0645\u06cc",
    "7": "\u062e\u0627\u0642\u0627\u0646\u06cc",
    "8": "\u0634\u0647\u0631\u06cc\u0627\u0631",
    "9": "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627": 9,
    "\u062d\u0627\u0641\u0638": 0,
    "\u062e\u0627\u0642\u0627\u0646\u06cc": 7,
    "\u0633\u0639\u062f\u06cc": 1,
    "\u0634\

In [19]:
#Defining our datasets and defining our data loaders over it
class Dataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, comments, targets=None, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = Dataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [20]:
#Defining our fine tuning model and loading our models initial parameters
class ClassificationModel(nn.Module):

    def __init__(self, config):
        super(ClassificationModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH,return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [21]:
#defining our model's evaluation metrics for learning and comparation
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)
    return y, loss

#defining a function to evaluate our model
def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss

#defining a function to process loss functions and do the forward and backward propagation
def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        loss.backward()
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
        optimizer.step()
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)
    return train_y, train_loss, step, eval_loss_min

In [22]:
#Finding predictions of our model
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    predictions = []
    prediction_probs = []

    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()
    return predictions, prediction_probs

In [38]:
#Using 5-fold cross validation to train our models and comparing each folds evaluation metrics
#Comparing 2 different learning rates for our models list
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np

EPOCHS = 60

for LEARNING_RATE in [2e-5, 2e-4]:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = 0
    fold = 1
    for train_index, test_index in kf.split(data):
        train, test = data.iloc[train_index], data.iloc[test_index]
        train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['Poet'])

        train = train.reset_index(drop=True)
        valid = valid.reset_index(drop=True)
        test = test.reset_index(drop=True)

        x_train, y_train = train['cleaned_poem'].values.tolist(), train['Poet'].values.tolist()
        x_valid, y_valid = valid['cleaned_poem'].values.tolist(), valid['Poet'].values.tolist()
        
        print(train.shape)
        print(valid.shape)
        print(test.shape)

        label_list = [data.iloc[30*j][1] for j in range(10)]
        train_data_loader = create_data_loader(train['cleaned_poem'].to_numpy(), train['Poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
        valid_data_loader = create_data_loader(valid['cleaned_poem'].to_numpy(), valid['Poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
        test_data_loader = create_data_loader(test['cleaned_poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pt_model = ClassificationModel(config=config)
        pt_model = pt_model.to(device)

        optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
        total_steps = len(train_data_loader) * EPOCHS
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )

        loss_fn = nn.CrossEntropyLoss()

        step = 0
        eval_loss_min = np.Inf
        history = collections.defaultdict(list)

        def eval_callback(epoch, epochs, output_path):
            def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
                statement = ''
                statement += 'Epoch: {}/{}...'.format(epoch, epochs)
                statement += 'Step: {}...'.format(step)

                statement += 'Train Loss: {:.6f}...'.format(train_loss)
                statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

                statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
                statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

                print(statement)

                if eval_loss <= eval_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                        eval_loss_min,
                        eval_loss))

                    torch.save(model.state_dict(), output_path)
                    eval_loss_min = eval_loss

                return eval_loss_min


            return eval_cb


        for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
            train_y, train_loss, step, eval_loss_min = train_op(
                model=pt_model,
                data_loader=train_data_loader,
                loss_fn=loss_fn,
                optimizer=optimizer,
                scheduler=scheduler,
                step=step,
                print_every_step=EEVERY_EPOCH,
                eval=True,
                eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
                eval_loss_min=eval_loss_min,
                eval_data_loader=valid_data_loader,
                clip=CLIP)

            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            eval_y, eval_loss = eval_op(
                model=pt_model,
                data_loader=valid_data_loader,
                loss_fn=loss_fn)

            eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

            history['train_acc'].append(train_score['acc'])
            history['train_loss'].append(train_loss)
            history['val_acc'].append(eval_score['acc'])
            history['val_loss'].append(eval_loss)
        test_comments = test['Poet'].to_numpy()
        preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)
        y_test, y_pred = [label_list.index(label) for label in test['Poet'].values], preds
        print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
        f1_scores += f1_score(y_test, y_pred, average="weighted")
        print()
        print(classification_report(y_test, y_pred))
    print("For learning rate = ", LEARNING_RATE, "average f1-score = ", f1_scores / 5)

(192, 2)
(48, 2)
(60, 2)


/tmp/ipykernel_826/3989587803.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_list = [data.iloc[30*j][1] for j in range(10)]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs... :   0%|          | 0/60 [00:00<?, ?it/s]

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

...

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

F1: 0.008333333333333333

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         9
           2       0.07      1.00      0.12         4
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         7

    accuracy                           0.07        60
   macro avg       0.01      0.10      0.01        60
weighted avg       0.00      0.07      0.01        60

For learning rate =  0.0002 average f1-score =  0.007619047619047618


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
#Picking another model for fine tuning and
#Loading our new model's tokenizer
MODEL_NAME_OR_PATH = "HooshvareLab/bert-fa-base-uncased-clf-persiannews"
OUTPUT_PATH = 'pytorch_model.bin'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "persiannews",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u062d\u0627\u0641\u0638",
    "1": "\u0633\u0639\u062f\u06cc",
    "2": "\u0645\u0648\u0644\u0648\u06cc",
    "3": "\u0646\u0627\u0635\u0631\u062e\u0633\u0631\u0648",
    "4": "\u0639\u0637\u0627\u0631",
    "5": "\u0641\u0631\u062f\u0648\u0633\u06cc",
    "6": "\u067e\u0631\u0648\u06cc\u0646 \u0627\u0639\u062a\u0635\u0627\u0645\u06cc",
    "7": "\u062e\u0627\u0642\u0627\u0646\u06cc",
    "8": "\u0634\u0647\u0631\u06cc\u0627\u0631",
    "9": "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627": 9,
    "\u062d\u0627\u0641\u0638": 0,
    "\u06

In [ ]:
#Using 5-fold cross validation to train our new models and comparing each folds evaluation metrics
#Comparing 2 different learning rates for our models list

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np

EPOCHS = 60

for LEARNING_RATE in [2e-5, 2e-4]:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = 0
    fold = 1
    for train_index, test_index in kf.split(data):
        train, test = data.iloc[train_index], data.iloc[test_index]
        train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['Poet'])

        train = train.reset_index(drop=True)
        valid = valid.reset_index(drop=True)
        test = test.reset_index(drop=True)

        x_train, y_train = train['cleaned_poem'].values.tolist(), train['Poet'].values.tolist()
        x_valid, y_valid = valid['cleaned_poem'].values.tolist(), valid['Poet'].values.tolist()
        
        print(train.shape)
        print(valid.shape)
        print(test.shape)

        label_list = [data.iloc[30*j][1] for j in range(10)]
        train_data_loader = create_data_loader(train['cleaned_poem'].to_numpy(), train['Poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
        valid_data_loader = create_data_loader(valid['cleaned_poem'].to_numpy(), valid['Poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
        test_data_loader = create_data_loader(test['cleaned_poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pt_model = ClassificationModel(config=config)
        pt_model = pt_model.to(device)
        
        optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
        total_steps = len(train_data_loader) * EPOCHS
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )

        loss_fn = nn.CrossEntropyLoss()

        step = 0
        eval_loss_min = np.Inf
        history = collections.defaultdict(list)

        def eval_callback(epoch, epochs, output_path):
            def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
                statement = ''
                statement += 'Epoch: {}/{}...'.format(epoch, epochs)
                statement += 'Step: {}...'.format(step)

                statement += 'Train Loss: {:.6f}...'.format(train_loss)
                statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

                statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
                statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

                print(statement)

                if eval_loss <= eval_loss_min:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                        eval_loss_min,
                        eval_loss))

                    torch.save(model.state_dict(), output_path)
                    eval_loss_min = eval_loss

                return eval_loss_min


            return eval_cb


        for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
            train_y, train_loss, step, eval_loss_min = train_op(
                model=pt_model,
                data_loader=train_data_loader,
                loss_fn=loss_fn,
                optimizer=optimizer,
                scheduler=scheduler,
                step=step,
                print_every_step=EEVERY_EPOCH,
                eval=True,
                eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
                eval_loss_min=eval_loss_min,
                eval_data_loader=valid_data_loader,
                clip=CLIP)

            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            eval_y, eval_loss = eval_op(
                model=pt_model,
                data_loader=valid_data_loader,
                loss_fn=loss_fn)

            eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

            history['train_acc'].append(train_score['acc'])
            history['train_loss'].append(train_loss)
            history['val_acc'].append(eval_score['acc'])
            history['val_loss'].append(eval_loss)
        test_comments = test['Poet'].to_numpy()
        preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)
        y_test, y_pred = [label_list.index(label) for label in test['Poet'].values], preds
        print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
        f1_scores += f1_score(y_test, y_pred, average="weighted")
        print()
        print(classification_report(y_test, y_pred))
    print("For learning rate = ", LEARNING_RATE, "average f1-score = ", f1_scores / 5)

(192, 2)
(48, 2)
(60, 2)


/tmp/ipykernel_826/3467170063.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_list = [data.iloc[30*j][1] for j in range(10)]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs... :   0%|          | 0/60 [00:00<?, ?it/s]

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

...

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

In [38]:
#Loading our cleaned data over to see if shortening our documents can help our model
short_data = pd.read_csv('/kaggle/working/cleaned.csv')
mn = 10000000
for i in range(short_data.shape[0]):
    mn = min(mn, len(short_data.iloc[i][1]))
print(mn)

1198


/tmp/ipykernel_34/2292410417.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mn = min(mn, len(short_data.iloc[i][1]))


In [39]:
#Choosing all document to be the same length of the shortest document
short_data['cleaned_poem'] = short_data['cleaned_poem'].str[:mn]

In [53]:
short_data = short_data[['cleaned_poem', 'Poet']]

In [65]:
#Learning our new documents with the same structure as before
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np


EPOCHS = 60
LEARNING_RATE = 2e-5
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = 'pytorch_model.bin'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

train, test = train_test_split(short_data, test_size=0.2, random_state=1, stratify=short_data['Poet'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['Poet'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['cleaned_poem'].values.tolist(), train['Poet'].values.tolist()
x_valid, y_valid = valid['cleaned_poem'].values.tolist(), valid['Poet'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

label_list = [short_data.iloc[30*j][1] for j in range(10)]
train_data_loader = create_data_loader(train['cleaned_poem'].to_numpy(), train['Poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['cleaned_poem'].to_numpy(), valid['Poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['cleaned_poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pt_model = ClassificationModel(config=config)
pt_model = pt_model.to(device)

optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)
test_comments = test['Poet'].to_numpy()
preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)
y_test, y_pred = [label_list.index(label) for label in test['Poet'].values], preds
print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred))

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u062d\u0627\u0641\u0638",
    "1": "\u0633\u0639\u062f\u06cc",
    "2": "\u0645\u0648\u0644\u0648\u06cc",
    "3": "\u0646\u0627\u0635\u0631\u062e\u0633\u0631\u0648",
    "4": "\u0639\u0637\u0627\u0631",
    "5": "\u0641\u0631\u062f\u0648\u0633\u06cc",
    "6": "\u067e\u0631\u0648\u06cc\u0646 \u0627\u0639\u062a\u0635\u0627\u0645\u06cc",
    "7": "\u062e\u0627\u0642\u0627\u0646\u06cc",
    "8": "\u0634\u0647\u0631\u06cc\u0627\u0631",
    "9": "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627": 9,
    "\u062d\u0627\u0641\u0638": 0,
    "\u062e\u0627\u0642\u0627\u0646\u06cc": 7,
    "\u0633\u0639\u062f\u06cc": 1,
    "\u0634\

/tmp/ipykernel_34/4118353974.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_list = [short_data.iloc[30*j][1] for j in range(10)]
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs... :   0%|          | 0/60 [00:00<?, ?it/s]

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

...

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

F1: 0.19166666666666668

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         6
           5       0.50      1.00      0.67         6
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         6
           8       0.14      1.00      0.25         6
           9       0.00      0.00      0.00         6

    accuracy                           0.30        60
   macro avg       0.16      0.30      0.19        60
weighted avg       0.16      0.30      0.19        60



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#defining a list of stop words and remove them from our original data
#to check if removing stop words can help our model
stop_words = ["و","در","به","از","كه","مي","اين","است","را","با","هاي","براي","آن","يك","شود","شده","خود","ها","كرد","شد","اي","تا","كند","بر","بود","گفت","نيز","وي","هم","كنند","دارد","ما","كرده","يا","اما","بايد","دو","اند","هر","خواهد","او","مورد","آنها","باشد","ديگر","مردم"]

def remove_stop_words(text):
    for stop_word in stop_words:
        text = text.replace(' ' + stop_word + ' ', ' ')
    return text

no_stop_data = pd.read_csv('/kaggle/working/cleaned.csv')
no_stop_data['cleaned_poem'] = no_stop_data['cleaned_poem'].apply(remove_stop_words)

no_stop_data = no_stop_data[['cleaned_poem', 'Poet']]

In [25]:
#Learning our new documents with the same structure as before
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import numpy as np


EPOCHS = 60
LEARNING_RATE = 2e-5
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = 'pytorch_model.bin'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

train, test = train_test_split(no_stop_data, test_size=0.2, random_state=1, stratify=no_stop_data['Poet'])
train, valid = train_test_split(train, test_size=0.2, random_state=1, stratify=train['Poet'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['cleaned_poem'].values.tolist(), train['Poet'].values.tolist()
x_valid, y_valid = valid['cleaned_poem'].values.tolist(), valid['Poet'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

label_list = [no_stop_data.iloc[30*j][1] for j in range(10)]
train_data_loader = create_data_loader(train['cleaned_poem'].to_numpy(), train['Poet'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['cleaned_poem'].to_numpy(), valid['Poet'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['cleaned_poem'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pt_model = ClassificationModel(config=config)
pt_model = pt_model.to(device)

optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)
test_comments = test['Poet'].to_numpy()
preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)
y_test, y_pred = [label_list.index(label) for label in test['Poet'].values], preds
print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred))

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u062d\u0627\u0641\u0638",
    "1": "\u0633\u0639\u062f\u06cc",
    "2": "\u0645\u0648\u0644\u0648\u06cc",
    "3": "\u0646\u0627\u0635\u0631\u062e\u0633\u0631\u0648",
    "4": "\u0639\u0637\u0627\u0631",
    "5": "\u0641\u0631\u062f\u0648\u0633\u06cc",
    "6": "\u067e\u0631\u0648\u06cc\u0646 \u0627\u0639\u062a\u0635\u0627\u0645\u06cc",
    "7": "\u062e\u0627\u0642\u0627\u0646\u06cc",
    "8": "\u0634\u0647\u0631\u06cc\u0627\u0631",
    "9": "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u06cc\u0631\u062c \u0645\u06cc\u0631\u0632\u0627": 9,
    "\u062d\u0627\u0641\u0638": 0,
    "\u062e\u0627\u0642\u0627\u0646\u06cc": 7,
    "\u0633\u0639\u062f\u06cc": 1,
    "\u0634\

/tmp/ipykernel_34/4043131811.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label_list = [no_stop_data.iloc[30*j][1] for j in range(10)]


pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epochs... :   0%|          | 0/60 [00:00<?, ?it/s]

...

Training... :   0%|          | 0/12 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

F1: 0.32857142857142857

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00         6
           7       1.00      1.00      1.00         6
           8       0.17      1.00      0.29         6
           9       0.00      0.00      0.00         6

    accuracy                           0.40        60
   macro avg       0.32      0.40      0.33        60
weighted avg       0.32      0.40      0.33        60



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#Drawing confusion matrix for last model
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[6 0 0 0 0 0 0 0 0 0]
 [0 6 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 6 0 0 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 0 6 0 0]
 [0 0 0 0 0 0 0 0 6 0]
 [0 0 0 0 0 0 0 0 6 0]]


In [3]:
#Import needed libraries
!pip install -q hazm
from hazm import word_tokenize, stopwords_list
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 

In [6]:
#Read and prepair data
def remove_stop_words(text):
    return [word for word in text if word not in stop_words]


df = pd.read_csv('/kaggle/input/document/cleaned.csv')
stop_words = stopwords_list()
tokenized_documents = [word_tokenize(document) for document in df["cleaned_poem"]]

for i, doc in enumerate(tokenized_documents):
    tokenized_documents[i] = remove_stop_words(doc)

In [7]:
#Define labels
# Define a dictionary to map poet names to labels
poet_label_mapping = {
    "حافظ": 0, 
    "سعدی": 1, 
    "مولوی": 2, 
    "ناصرخسرو": 3, 
    "عطار": 4, 
    "فردوسی": 5, 
    "پروین اعتصامی": 6, 
    "خاقانی": 7, 
    "شهریار": 8, 
    "ایرج میرزا": 9
}

# Map poet names to labels
labels = [poet_label_mapping[poet] for poet in df["Poet"]]

In [8]:
final_docs = [' '.join(doc) for doc in tokenized_documents]  # convert documents to strings without stopwords

In [9]:
#Train, test split
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(final_docs, labels, test_size=0.2, random_state=50)

In [11]:
#Random Forest
# Feature Extraction
tfidf_vectorizer = TfidfVectorizer(max_features=1000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

# Evaluate the Model
rf_predictions = rf_classifier.predict(X_test_tfidf)

# Calculate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Accuracy:", rf_accuracy)

# Generate classification report
print(classification_report(y_test, rf_predictions))
cf_matrix_rf = confusion_matrix(y_test, rf_predictions)
print(cf_matrix_rf)

Accuracy: 0.8666666666666667
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.83      1.00      0.91         5
           2       1.00      0.80      0.89        10
           3       0.83      1.00      0.91         5
           4       1.00      0.83      0.91         6
           5       0.83      1.00      0.91         5
           6       0.50      1.00      0.67         4
           7       1.00      0.89      0.94         9
           8       1.00      1.00      1.00         4
           9       0.00      0.00      0.00         4

    accuracy                           0.87        60
   macro avg       0.80      0.85      0.81        60
weighted avg       0.86      0.87      0.85        60

[[8 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 8 1 0 0 0 0 0 1]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 5 0 1 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 1 8 0 0]
 [0 0 0 0 0 0 0 0 4 0]
 [0 1

In [12]:
#SVM
svm_classifier = SVC(kernel='linear') 
svm_classifier.fit(X_train_tfidf, y_train)
svm_predictions = svm_classifier.predict(X_test_tfidf)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, svm_predictions)
print("Accuracy:", accuracy_svm)

# Generate classification report
print(classification_report(y_test, svm_predictions))
cf_matrix_svm = confusion_matrix(y_test, svm_predictions)
print(cf_matrix_svm)

Accuracy: 0.9166666666666666
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.83      1.00      0.91         5
           2       1.00      0.90      0.95        10
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         5
           6       0.80      1.00      0.89         4
           7       1.00      0.78      0.88         9
           8       1.00      1.00      1.00         4
           9       0.50      0.75      0.60         4

    accuracy                           0.92        60
   macro avg       0.91      0.93      0.92        60
weighted avg       0.94      0.92      0.92        60

[[7 1 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 9 0 0 0 0 0 0 1]
 [0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 6 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 7 0 2]
 [0 0 0 0 0 0 0 0 4 0]
 [0 0